<a href="https://colab.research.google.com/github/dobrevajovana/Makedonizer/blob/main/NaturalLanguageinference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 44.2 MB/s 
     |████████████████████████████████| 6.5 MB 33.7 MB/s 
     |████████████████████████████████| 895 kB 43.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.5 MB/s 


In [ ]:
import pandas as pd
import re
import torch
import time
import random

from torch.utils.data import Dataset, TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pad_sequence

from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup, XLMRobertaTokenizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report, matthews_corrcoef, f1_score

import pickle
import os
import numpy as np

from tqdm.notebook import tqdm

In [ ]:
TOKENIZER_PATH = '/content/drive/MyDrive/Macedonizer-Evaluation/mk-roberta-tokenizer'
MODEL_PATH = '/content/drive/MyDrive/macedonizer.finki/mk-roberta-common-crawl/checkpoint-3800000'

In [ ]:
# TOKENIZER_PATH = 'xlm-roberta-base'
# MODEL_PATH = 'xlm-roberta-base'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Macedonizer-Evaluation/NLI-Dataset/SNLI-Dataset/train.csv')
test_df   = pd.read_csv('/content/drive/MyDrive/Macedonizer-Evaluation/NLI-Dataset/SNLI-Dataset/test.csv')

In [ ]:
train_df.label.unique()

array([ 1,  2,  0, -1])

In [ ]:
train_df = train_df[train_df['label'] != -1]
test_df = test_df[test_df['label'] != -1]

In [ ]:
train_df.label.unique()

array([1, 2, 0])

In [ ]:
train_df = train_df.sample(frac=1).reset_index(drop=True)
test_df = test_df.sample(frac=1).reset_index(drop=True)

In [ ]:
class NLIDataset(Dataset):

  def __init__(self, train_df, test_df, tokenizer_path):

    self.train_df = train_df
    self.test_df = test_df

    self.tokenizer = RobertaTokenizer.from_pretrained(tokenizer_path, do_lower_case=True)
    # self.tokenizer = XLMRobertaTokenizer.from_pretrained(tokenizer_path, do_lower_case=True)

    self.train_data = None
    self.test_data = None
    self.init_data()

  def init_data(self):
    self.train_data = self.load_data(self.train_df)
    self.test_data = self.load_data(self.test_df)

  def load_data(self, df):
    MAX_LEN = 250
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['premise_mk'].to_list()
    hypothesis_list = df['hypothesis_mk'].to_list()
    label_list = df['label'].to_list()

    for (premise, hypothesis, label) in zip(premise_list, hypothesis_list, label_list):
      premise_id = self.tokenizer.encode(premise, add_special_tokens = False, truncation=True, max_length=MAX_LEN)
      hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False, truncation=True, max_length=MAX_LEN)
      pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]

      premise_len = len(premise_id)
      hypothesis_len = len(hypothesis_id)

      # segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))
      attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))
      token_ids.append(torch.tensor(pair_token_ids))
      # seg_ids.append(segment_ids)
      mask_ids.append(attention_mask_ids)
      y.append(label)

    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    # seg_ids = pad_sequence(seg_ids, batch_first=True)
    y = torch.tensor(y)

    dataset = TensorDataset(token_ids, mask_ids, y) # seg_ids,
    print(len(dataset))
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    train_loader = DataLoader(
      self.train_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    val_loader = DataLoader(
      self.test_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    return train_loader, val_loader

In [ ]:
dataset = NLIDataset(train_df, test_df, tokenizer_path=TOKENIZER_PATH)

50946
9824


In [ ]:
train_loader, test_loader = dataset.get_data_loaders(batch_size=16)

In [ ]:
# This cell is for XLM-RoBERTa-Base Fine Tuning
# uncomment if you want to fine-tune this model

# from transformers import XLMRobertaForSequenceClassification

# EPOCHS = 10

# model = XLMRobertaForSequenceClassification.from_pretrained(MODEL_PATH, num_labels=3)
# model.to(device)

In [ ]:
# This cell is for our RoBERTa-Base Fine Tuning
# uncomment if you want to fine-tune this model

EPOCHS = 10

model = RobertaForSequenceClassification.from_pretrained(MODEL_PATH, num_labels=3)
model.to(device)

Some weights of the model checkpoint at /content/drive/MyDrive/macedonizer.finki/mk-roberta-common-crawl/checkpoint-3800000 were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/macedonizer.finki/mk

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [ ]:
LEARNING_RATE = 1e-05

optimizer = AdamW(
    model.parameters(),
    lr=LEARNING_RATE,
    eps=1e-8
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = len(train_loader) * EPOCHS
  )

In [ ]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
# RoBERTa-Base-All
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 124,647,939 trainable parameters


In [ ]:
# XLM-RoBERTa-Base
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 278,045,955 trainable parameters


In [ ]:
def single_model_performance(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  print("Confusion Matrix:")
  print(confusion_matrix(y_true=labels_flat, y_pred=preds_flat))
  print(classification_report(y_true=labels_flat, y_pred=preds_flat))
  print(f'MCC score: {matthews_corrcoef(y_true=labels_flat, y_pred=preds_flat)}')

def matthews_corrcoef_func(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return matthews_corrcoef(labels_flat, preds_flat)

def recall_score_func(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return recall_score(labels_flat, preds_flat, average='weighted')

def precision_score_func(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return precision_score(labels_flat, preds_flat, average='weighted')

def accuracy_score_func(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return accuracy_score(labels_flat, preds_flat)

def f1_score_func(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
def evaluate(test_loader):
  model.eval()

  loss_val_total = 0
  predictions, true_vals = [], []

  for batch in tqdm(test_loader):

    batch = tuple(b.to(device) for b in batch)

    inputs = {'input_ids':      batch[0],
              'attention_mask': batch[1],
              'labels':         batch[2],
              }

    with torch.no_grad():
        outputs = model(**inputs)

    loss = outputs[0]
    logits = outputs[1]
    loss_val_total += loss.item()

    logits = logits.detach().cpu().numpy()
    label_ids = inputs['labels'].cpu().numpy()
    predictions.append(logits)
    true_vals.append(label_ids)

  loss_val_avg = loss_val_total/len(test_loader)

  predictions = np.concatenate(predictions, axis=0)
  true_vals = np.concatenate(true_vals, axis=0)

  return loss_val_avg, predictions, true_vals

In [ ]:
def train(model, train_loader, test_loader, optimizer, scheduler):
  total_step = len(train_loader)

  for epoch in range(EPOCHS):
    start = time.time()
    model.train()

    loss_train_total = 0

    progress_bar = tqdm(train_loader,
                      desc='Epoch {:1d}'.format(epoch),
                      leave=False,
                      disable=False)

    for batch in progress_bar:

      model.zero_grad()
      batch = tuple(b.to(device) for b in batch)
      inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }

      outputs = model(**inputs)
      loss = outputs[0]
      loss_train_total += loss.item()
      loss.backward()

      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      optimizer.step()
      scheduler.step()

      progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    loss_train_avg = loss_train_total/len(train_loader)
    print(f'Epoch: {epoch}')
    tqdm.write(f'Training loss: {loss_train_avg}')

    # Evaluation
    val_loss, predictions, true_vals = evaluate(test_loader)

    val_f1 = f1_score_func(predictions, true_vals)
    print(f'Validation loss: {val_loss}')
    print(f'F1 Score (weighted): {val_f1}')
    print(f'Accuracy Score: {accuracy_score_func(predictions, true_vals)}')
    print(f'Precision Score: {precision_score_func(predictions, true_vals)}')
    print(f'Recall Score: {recall_score_func(predictions, true_vals)}')
    print(f'MCC Score: {matthews_corrcoef_func(predictions, true_vals)}')
    single_model_performance(predictions, true_vals)
    print('')

## SNLI Dataset on RoBERTa-CommonCrawl

In [ ]:
# SNLI on our RoBERTa-CommonCrawl model
# Update on 26.03.2022

train(model, train_loader, test_loader, optimizer, scheduler)

Epoch 0:   0%|          | 0/3185 [00:00<?, ?it/s]

Epoch: 0
Training loss: 0.7829976291212968


  0%|          | 0/614 [00:00<?, ?it/s]

Validation loss: 0.6153911024040819
F1 Score (weighted): 0.7544641627890181
Accuracy Score: 0.7543770358306189
Precision Score: 0.7570683160109847
Recall Score: 0.7543770358306189
MCC Score: 0.6323967544433321
Confusion Matrix:
[[2724  468  176]
 [ 429 2390  400]
 [ 359  581 2297]]
              precision    recall  f1-score   support

           0       0.78      0.81      0.79      3368
           1       0.69      0.74      0.72      3219
           2       0.80      0.71      0.75      3237

    accuracy                           0.75      9824
   macro avg       0.76      0.75      0.75      9824
weighted avg       0.76      0.75      0.75      9824

MCC score: 0.6323967544433321



Epoch 1:   0%|          | 0/3185 [00:00<?, ?it/s]

...

Epoch 5:   0%|          | 0/3185 [00:00<?, ?it/s]

Epoch: 5
Training loss: 0.26737043040788483


  0%|          | 0/614 [00:00<?, ?it/s]

Validation loss: 0.8357256421241424
F1 Score (weighted): 0.780677311118476
Accuracy Score: 0.7809446254071661
Precision Score: 0.7810596719077794
Recall Score: 0.7809446254071661
MCC Score: 0.6716797871211283
Confusion Matrix:
[[2709  417  242]
 [ 371 2313  535]
 [ 204  383 2650]]
              precision    recall  f1-score   support

           0       0.82      0.80      0.81      3368
           1       0.74      0.72      0.73      3219
           2       0.77      0.82      0.80      3237

    accuracy                           0.78      9824
   macro avg       0.78      0.78      0.78      9824
weighted avg       0.78      0.78      0.78      9824

MCC score: 0.6716797871211283



Epoch 6:   0%|          | 0/3185 [00:00<?, ?it/s]

## SNLI Dataset on XLM-RoBERTa-Model

In [ ]:
# SNLI on XLM-RoBERTa-Base model
train(model, train_loader, test_loader, optimizer, scheduler)

Epoch 0:   0%|          | 0/3185 [00:00<?, ?it/s]

Epoch: 0
Training loss: 1.0068349694906262


  0%|          | 0/614 [00:00<?, ?it/s]

Validation loss: 0.9183981688480812
F1 Score (weighted): 0.5571119862642198
Confusion Matrix:
[[2289  357  722]
 [ 787 1052 1380]
 [ 572  400 2265]]
              precision    recall  f1-score   support

           0       0.63      0.68      0.65      3368
           1       0.58      0.33      0.42      3219
           2       0.52      0.70      0.60      3237

    accuracy                           0.57      9824
   macro avg       0.58      0.57      0.56      9824
weighted avg       0.58      0.57      0.56      9824

MCC score: 0.36483712893391657



Epoch 1:   0%|          | 0/3185 [00:00<?, ?it/s]

...

Validation loss: 0.8952408880238036
F1 Score (weighted): 0.6589695636446548
Confusion Matrix:
[[2345  475  548]
 [ 532 1814  873]
 [ 395  516 2326]]
              precision    recall  f1-score   support

           0       0.72      0.70      0.71      3368
           1       0.65      0.56      0.60      3219
           2       0.62      0.72      0.67      3237

    accuracy                           0.66      9824
   macro avg       0.66      0.66      0.66      9824
weighted avg       0.66      0.66      0.66      9824

MCC score: 0.4918001970877914



Epoch 9:   0%|          | 0/3185 [00:00<?, ?it/s]

Epoch: 9
Training loss: 0.5573024438141466


  0%|          | 0/614 [00:00<?, ?it/s]

Validation loss: 0.9085395886329175
F1 Score (weighted): 0.6622434205984314
Confusion Matrix:
[[2387  537  444]
 [ 557 1938  724]
 [ 430  623 2184]]
              precision    recall  f1-score   support

           0       0.71      0.71      0.71      3368
           1       0.63      0.60      0.61      3219
           2       0.65      0.67      0.66      3237

    accuracy                           0.66      9824
   macro avg       0.66      0.66      0.66      9824
weighted avg       0.66      0.66      0.66      9824

MCC score: 0.49382131443163224



In [ ]:
assert True == False

AssertionError: ignored

## SNLI Dataset On Our RoBERTa Model

In [ ]:
# SNLI on our RoBERTa-Base model
# Update on 08.03.2022

# Run this cell!
train(model, train_loader, test_loader, optimizer, scheduler)

Epoch 0:   0%|          | 0/3185 [00:00<?, ?it/s]

Epoch: 0
Training loss: 0.7878371387860464


  0%|          | 0/614 [00:00<?, ?it/s]

Validation loss: 0.6369077361610502
F1 Score (weighted): 0.7363613026454001
Accuracy Score: 0.7375814332247557
Precision Score: 0.7394583806162941
Recall Score: 0.7375814332247557
MCC Score: 0.6076960516218738
Confusion Matrix:
[[2791  355  222]
 [ 560 2251  408]
 [ 475  558 2204]]
              precision    recall  f1-score   support

           0       0.73      0.83      0.78      3368
           1       0.71      0.70      0.71      3219
           2       0.78      0.68      0.73      3237

    accuracy                           0.74      9824
   macro avg       0.74      0.74      0.74      9824
weighted avg       0.74      0.74      0.74      9824

MCC score: 0.6076960516218738



Epoch 1:   0%|          | 0/3185 [00:00<?, ?it/s]

...

Validation loss: 1.137490622596876
F1 Score (weighted): 0.7712251133345844
Accuracy Score: 0.7708672638436482
Precision Score: 0.7718358985586519
Recall Score: 0.7708672638436482
MCC Score: 0.6564242541438758
Confusion Matrix:
[[2660  449  259]
 [ 363 2384  472]
 [ 233  475 2529]]
              precision    recall  f1-score   support

           0       0.82      0.79      0.80      3368
           1       0.72      0.74      0.73      3219
           2       0.78      0.78      0.78      3237

    accuracy                           0.77      9824
   macro avg       0.77      0.77      0.77      9824
weighted avg       0.77      0.77      0.77      9824

MCC score: 0.6564242541438758



Epoch 9:   0%|          | 0/3185 [00:00<?, ?it/s]

Epoch: 9
Training loss: 0.1715386151632563


  0%|          | 0/614 [00:00<?, ?it/s]

Validation loss: 1.1757785232998552
F1 Score (weighted): 0.7719810410002043
Accuracy Score: 0.7717833876221498
Precision Score: 0.7722407014790428
Recall Score: 0.7717833876221498
MCC Score: 0.6576701889491979
Confusion Matrix:
[[2693  427  248]
 [ 383 2371  465]
 [ 244  475 2518]]
              precision    recall  f1-score   support

           0       0.81      0.80      0.81      3368
           1       0.72      0.74      0.73      3219
           2       0.78      0.78      0.78      3237

    accuracy                           0.77      9824
   macro avg       0.77      0.77      0.77      9824
weighted avg       0.77      0.77      0.77      9824

MCC score: 0.6576701889491979



In [ ]:
# SNLI on our RoBERTa-Base model
train(model, train_loader, test_loader, optimizer, scheduler)

Epoch 0:   0%|          | 0/3185 [00:00<?, ?it/s]

Epoch: 0
Training loss: 0.7894805244972792


  0%|          | 0/614 [00:00<?, ?it/s]

Validation loss: 0.6359457686446389
F1 Score (weighted): 0.7377085092055004
Confusion Matrix:
[[2793  325  250]
 [ 581 2106  532]
 [ 460  412 2365]]
              precision    recall  f1-score   support

           0       0.73      0.83      0.78      3368
           1       0.74      0.65      0.69      3219
           2       0.75      0.73      0.74      3237

    accuracy                           0.74      9824
   macro avg       0.74      0.74      0.74      9824
weighted avg       0.74      0.74      0.74      9824

MCC score: 0.6103734650769178



Epoch 1:   0%|          | 0/3185 [00:00<?, ?it/s]

...

Validation loss: 1.1492493315486099
F1 Score (weighted): 0.7697040800788267
Confusion Matrix:
[[2681  413  274]
 [ 374 2326  519]
 [ 230  453 2554]]
              precision    recall  f1-score   support

           0       0.82      0.80      0.81      3368
           1       0.73      0.72      0.73      3219
           2       0.76      0.79      0.78      3237

    accuracy                           0.77      9824
   macro avg       0.77      0.77      0.77      9824
weighted avg       0.77      0.77      0.77      9824

MCC score: 0.6545536893592653



Epoch 9:   0%|          | 0/3185 [00:00<?, ?it/s]

Epoch: 9
Training loss: 0.16485650994642412


  0%|          | 0/614 [00:00<?, ?it/s]

Validation loss: 1.199278458062535
F1 Score (weighted): 0.7696899084257834
Confusion Matrix:
[[2672  448  248]
 [ 374 2354  491]
 [ 236  468 2533]]
              precision    recall  f1-score   support

           0       0.81      0.79      0.80      3368
           1       0.72      0.73      0.73      3219
           2       0.77      0.78      0.78      3237

    accuracy                           0.77      9824
   macro avg       0.77      0.77      0.77      9824
weighted avg       0.77      0.77      0.77      9824

MCC score: 0.6542147152477191



## Earlier Dataset on our RoBERTa Model

In [ ]:
# Our RoBERTa-Base-All
train(model, train_loader, test_loader, optimizer, scheduler)

Epoch 0:   0%|          | 0/1444 [00:00<?, ?it/s]

Epoch: 0
Training loss: 0.41131512911015106


  0%|          | 0/133 [00:00<?, ?it/s]

Validation loss: 0.4366325318701285
F1 Score (weighted): 0.8055018196043585
Confusion Matrix:
[[ 654  188]
 [ 227 1057]]
              precision    recall  f1-score   support

           0       0.74      0.78      0.76       842
           2       0.85      0.82      0.84      1284

    accuracy                           0.80      2126
   macro avg       0.80      0.80      0.80      2126
weighted avg       0.81      0.80      0.81      2126

MCC score: 0.595617015298504



Epoch 1:   0%|          | 0/1444 [00:00<?, ?it/s]

...

Validation loss: 0.7807049735946426
F1 Score (weighted): 0.8164481495304896
Confusion Matrix:
[[ 623  219]
 [ 169 1115]]
              precision    recall  f1-score   support

           0       0.79      0.74      0.76       842
           2       0.84      0.87      0.85      1284

    accuracy                           0.82      2126
   macro avg       0.81      0.80      0.81      2126
weighted avg       0.82      0.82      0.82      2126

MCC score: 0.6153258993804965



Epoch 4:   0%|          | 0/1444 [00:00<?, ?it/s]

Epoch: 4
Training loss: 0.09441861466653297


  0%|          | 0/133 [00:00<?, ?it/s]

Validation loss: 0.9746032917304874
F1 Score (weighted): 0.8188901536839144
Confusion Matrix:
[[ 662  180]
 [ 206 1078]]
              precision    recall  f1-score   support

           0       0.76      0.79      0.77       842
           2       0.86      0.84      0.85      1284

    accuracy                           0.82      2126
   macro avg       0.81      0.81      0.81      2126
weighted avg       0.82      0.82      0.82      2126

MCC score: 0.622680132496728



## Earlied Dataset on XLM-RoBERTa-Base

In [ ]:
# XLM-RoBERTa-Base
train(model, train_loader, test_loader, optimizer, scheduler)

Epoch 0:   0%|          | 0/1444 [00:00<?, ?it/s]

Epoch: 0
Training loss: 0.3904455008508942


  0%|          | 0/133 [00:00<?, ?it/s]

Validation loss: 0.39346628102536935
F1 Score (weighted): 0.8567935939656646
Confusion Matrix:
[[ 617  225]
 [  73 1211]]
              precision    recall  f1-score   support

           0       0.89      0.73      0.81       842
           2       0.84      0.94      0.89      1284

    accuracy                           0.86      2126
   macro avg       0.87      0.84      0.85      2126
weighted avg       0.86      0.86      0.86      2126

MCC score: 0.7060502852050432



Epoch 1:   0%|          | 0/1444 [00:00<?, ?it/s]

Epoch: 1
Training loss: 0.2424309109092543


  0%|          | 0/133 [00:00<?, ?it/s]

In [ ]:
# https://huggingface.co/datasets/newsph_nli
# GOOGLETRANSLATE excel

In [ ]:
# https://huggingface.co/datasets/snli

In [ ]:
# def train(model, train_loader, test_loader, optimizer, scheduler):
#   total_step = len(train_loader)

#   for epoch in range(EPOCHS):
#     start = time.time()
#     model.train()

#     loss_train_total = 0

#     progress_bar = tqdm(train_loader,
#                       desc='Epoch {:1d}'.format(epoch),
#                       leave=False,
#                       disable=False)

#     for batch in progress_bar:

#       model.zero_grad()
#       batch = tuple(b.to(device) for b in batch)
#       inputs = {
#             'input_ids': batch[0],
#             'attention_mask': batch[1],
#             'labels': batch[2]
#         }

#       outputs = model(**inputs)
#       loss = outputs[0]
#       loss_train_total += loss.item()
#       loss.backward()

#       torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

#       optimizer.step()
#       scheduler.step()

#       progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

#     loss_train_avg = loss_train_total/len(train_loader)
#     print(f'Epoch: {epoch}')
#     tqdm.write(f'Training loss: {loss_train_avg}')

#     # Evaluation
#     val_loss, predictions, true_vals = evaluate(test_loader)

#     val_f1 = f1_score_func(predictions, true_vals)
#     print(f'Validation loss: {val_loss}')
#     print(f'F1 Score (weighted): {val_f1}')
#     single_model_performance(predictions, true_vals)
#     print('')

true i predicted kolonite
suma=0
for t,p in zip(true,predicted):

  for i,j in zip(t,p):
    if i==j:
       count+=1
    suma+=1
count/vkupenbrojtagovi